In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,"((, when, she, thought, it, over, afterwards, ...",Carroll


In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,continue,terror,sh,position,head,peer,wildly,avoid,chatte,fatigue,...,grammar,rome,wonderland,applause,fairy,formal,apparently,dark,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll


In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9897209985315712

Test set score: 0.9151982378854625


In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2724, 3020) (2724,)
Training set score: 0.9636563876651982

Test set score: 0.9355726872246696


In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9196035242290749

Test set score: 0.9091409691629956


In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.7034666666666667


col_0,Austen,Carroll
row_0,,
Austen,1158,21
Carroll,535,161


## Challenge 0:
Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

In [15]:
#trying svc
from sklearn.svm import SVC

svm = SVC()
train = svm.fit(X_train, y_train)

print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

Training set score: 0.7452276064610867

Test set score: 0.7340308370044053


In [16]:
word_counts

,continue,terror,sh,position,head,peer,wildly,avoid,chatte,fatigue,...,grammar,rome,wonderland,applause,fairy,formal,apparently,dark,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(In, another, moment, down, went, Alice, after...",Carroll
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, rabbit, -, hole, went, straight, on, lik...",Carroll
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Either, the, well, was, very, deep, ,, or, sh...",Carroll
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"(First, ,, she, tried, to, look, down, and, ma...",Carroll
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(She, took, down, a, jar, from, one, of, the, ...",Carroll


In [17]:
#adding word counts
word_counts.iloc[:, 0:3020] = word_counts.iloc[:, 0:3020].astype(str).astype(int)
word_counts['WordCount'] = word_counts.sum(axis=1)
word_counts.iloc[:, 0:3020] = word_counts.iloc[:, 0:3020].astype(str).astype(int)
word_counts['WordCount'] = word_counts.sum(axis=1)
word_counts['WordCount'].describe()

count    4540.000000
mean        9.024009
std         8.170123
min         0.000000
25%         3.000000
50%         7.000000
75%        12.000000
max        75.000000
Name: WordCount, dtype: float64

In [18]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.98568281938326

Test set score: 0.9256607929515418


In [19]:
train = lr.fit(X_train, y_train)
print('Training set score:', lr.score(X_train, y_train))
print('Test set score:', lr.score(X_test, y_test))

train = clf.fit(X_train, y_train)
print('\nTraining set score:', clf.score(X_train, y_train))
print('Test set score:', clf.score(X_test, y_test))

Training set score: 0.9636563876651982
Test set score: 0.933920704845815

Training set score: 0.920704845814978
Test set score: 0.9080396475770925


In [20]:
train = svm.fit(X_train, y_train)

print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

Training set score: 0.7448604992657856

Test set score: 0.7340308370044053


In [21]:
#including sentence length of the previous and following sentences
word_counts['preceding'] = word_counts.WordCount.shift(-1)
word_counts['following'] = word_counts.WordCount.shift(1)
word_counts = word_counts.fillna(0)

In [22]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)
print('Training set score:', rfc.score(X_train, y_train))
print('Test set score:', rfc.score(X_test, y_test))

train = lr.fit(X_train, y_train)
print('\nTraining set score:', lr.score(X_train, y_train))
print('Test set score:', lr.score(X_test, y_test))

train = clf.fit(X_train, y_train)
print('\nTraining set score:', clf.score(X_train, y_train))
print('Test set score:', clf.score(X_test, y_test))

train = svm.fit(X_train, y_train)

print('\nTraining set score:', svm.score(X_train, y_train))
print('Test set score:', svm.score(X_test, y_test))

Training set score: 0.9911894273127754
Test set score: 0.9179515418502202

Training set score: 0.9651248164464024
Test set score: 0.9322687224669604

Training set score: 0.919970631424376
Test set score: 0.9063876651982379

Training set score: 0.7448604992657856
Test set score: 0.7340308370044053


In [23]:
all_tags = [w.pos_ for w in alice_doc]
newlist = set(all_tags)
newlist

{'ADJ',
 'ADP',
 'ADV',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'VERB',
 'X'}

In [24]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.pos_
                for token in text]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=newlist)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, newlist] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.pos_
                 for token in sentence]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [25]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,NOUN,CCONJ,DET,PROPN,PRON,X,PUNCT,NUM,ADP,VERB,INTJ,ADJ,PART,ADV,text_sentence,text_source
0,12,6,5,2,3,0,9,0,8,13,0,3,3,3,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,8,2,6,2,4,0,7,0,8,11,0,7,1,7,"(So, she, was, considering, in, her, own, mind...",Carroll
2,2,1,3,4,2,0,4,0,6,5,2,2,1,1,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,1,0,3,0,0,2,2,1,0,0,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,15,8,11,4,13,0,17,0,18,19,0,5,5,11,"((, when, she, thought, it, over, afterwards, ...",Carroll


In [26]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)
print('Training set score:', rfc.score(X_train, y_train))
print('Test set score:', rfc.score(X_test, y_test))

train = lr.fit(X_train, y_train)
print('\nTraining set score:', lr.score(X_train, y_train))
print('Test set score:', lr.score(X_test, y_test))

train = clf.fit(X_train, y_train)
print('\nTraining set score:', clf.score(X_train, y_train))
print('Test set score:', clf.score(X_test, y_test))

train = svm.fit(X_train, y_train)

print('\nTraining set score:', svm.score(X_train, y_train))
print('Test set score:', svm.score(X_test, y_test))

Training set score: 0.973935389133627
Test set score: 0.7830396475770925

Training set score: 0.7878120411160059
Test set score: 0.789647577092511

Training set score: 0.829295154185022
Test set score: 0.7918502202643172

Training set score: 0.8571953010279001
Test set score: 0.7857929515418502


## Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [27]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [28]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [29]:
#persuasion = gutenberg.raw('austen-persuasion.txt')
brown = gutenberg.raw('chesterton-brown.txt')
brown = text_cleaner(brown)

In [30]:
brown_doc = nlp(brown)

In [31]:
# Group into sentences.
brown_sents = [[sent, "Chesterton"] for sent in brown_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
brown_sents = brown_sents[0:len(alice_sents)]

In [33]:
brownwords = bag_of_words(brown_doc)
common_words = set(alicewords + brownwords)

In [34]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
brown_sentences = pd.DataFrame(brown_sents)
brown_bow = bow_features(brown_sentences, common_words)

Processing row 0
Processing row 500
Processing row 1000


In [35]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + brown_sents)
word_counts = bow_features(sentences, common_words)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000


In [36]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [37]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_b_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    brown_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_b_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Chesterton'] * brown_bow.shape[0])])

In [44]:
# Model.
lr = LogisticRegression()
train = lr.fit(X_train, y_train)

print('\nTest set score:', lr.score(X_b_test, y_b_test))
lr_b_predicted = lr.predict(X_b_test)
pd.crosstab(y_b_test, lr_b_predicted)


Test set score: 0.7665245202558635


col_0,Carroll,Chesterton
row_0,,
Carroll,359,338
Chesterton,100,1079
